In [1]:
from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras import layers
import numpy as np
from keras.callbacks import TensorBoard

In [2]:
input_tensor = layers.Input(shape=(224, 224, 3))
base_model = keras.applications.MobileNetV2(input_tensor=input_tensor,weights='imagenet',include_top=False) 
x = base_model.output
for layer in base_model.layers:
    layer.trainable = False
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.6)(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dense(1024, activation='relu')(x)
predictions = layers.Dense(2, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics='accuracy')

In [4]:
img_datagen = image.ImageDataGenerator(rescale=1./255,validation_split = 0.2)

train_ds = img_datagen.flow_from_directory(
    directory='d:/bioinfo/DLSPseq/ref_information/train_Data/train/',
    class_mode='categorical',
    batch_size=32,
    target_size=(224, 224),
    subset = 'training')
validation_ds = img_datagen.flow_from_directory(
    directory='d:/bioinfo/DLSPseq/ref_information/train_Data/train/',
    class_mode='categorical',
    batch_size=32,
    target_size=(224, 224),
    subset = 'validation')


Found 1708 images belonging to 2 classes.
Found 426 images belonging to 2 classes.


In [5]:
callbacks = TensorBoard(log_dir='d:/bioinfo/DLSPseq/result/epoch100',histogram_freq=1,embeddings_freq=1)
model.fit(train_ds, epochs=100, steps_per_epoch = train_ds.n/train_ds.batch_size, validation_data=validation_ds,callbacks=[callbacks])

Epoch 1/100
53/53 [==============================] - 17s 195ms/step - loss: 0.7344 - accuracy: 0.6944 - val_loss: 0.4739 - val_accuracy: 0.8615
Epoch 2/100
53/53 [==============================] - 8s 141ms/step - loss: 0.5733 - accuracy: 0.7278 - val_loss: 0.3714 - val_accuracy: 0.8873
Epoch 3/100
54/53 [==============================] - ETA: 0s - loss: 0.5582 - accuracy: 0.7359

In [ ]:
model_path = "D:/bioinfo/DLSPseq/data/V2_2_normal_epoch16/"
model = keras.models.load_model(model_path)
model = Model(inputs = model.input,outputs = model.get_layer('global_average_pooling2d_1').output)
img_path = "D:/bioinfo/DLSPseq/data/norm_img/test1000/TCGA-OL-A5RW-01Z-00-DX1/8801_27201.jpg"
img = image.load_img(img_path,target_size = (224,224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
features = model.predict(x)